
## Brian's Week 3 Homework for Coursera Capstone part 1 ##

scrape the Wikipedia page

In [1]:
#scrape table on wikipedia page
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [2]:
len(tr_elements)

294

In [3]:

#print('parsing start')
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
   # print('parsing', j)
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        data
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
print('done parsing table')        

done parsing table


In [4]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


Looks like we have the columns in the wrong order, so let's fix that now

In [7]:
#looks like we have the columns in the wrong order, so fix that

df = df[['Postcode', 'Borough', 'Neighbourhood\n']]#notice the \n in the third column - that took me a while to figure out
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


rename the columns to match the correct project name requirements

In [8]:
#rename columns to the project requirements
df.rename({'Neighbourhood\n': 'Neighborhood'}, axis='columns', inplace=True)
df.rename({'Postcode': 'PostalCode'}, axis='columns', inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Clean up the dataframe

In [9]:
#clean up df and keep the old one
df_unclean=df
df.shape
#take out the newlines in the data
df = df.replace('\n','', regex=True)
#lets look at the values we have and see that they make sense
print(df['Neighborhood'].unique())
print(df['Borough'].unique())
print(df['PostalCode'].unique())

['Not assigned' 'Parkwoods' 'Victoria Village' 'Harbourfront' 'Regent Park'
 'Lawrence Heights' 'Lawrence Manor' 'Islington Avenue' 'Rouge' 'Malvern'
 'Don Mills North' 'Woodbine Gardens' 'Parkview Hill' 'Ryerson'
 'Garden District' 'Glencairn' 'Cloverdale' 'Islington' 'Martin Grove'
 'Princess Gardens' 'West Deane Park' 'Highland Creek' 'Rouge Hill'
 'Port Union' 'Flemingdon Park' 'Don Mills South' 'Woodbine Heights'
 'St. James Town' 'Humewood-Cedarvale' 'Bloordale Gardens' 'Eringate'
 'Markland Wood' 'Old Burnhamthorpe' 'Guildwood' 'Morningside' 'West Hill'
 'The Beaches' 'Berczy Park' 'Caledonia-Fairbanks' 'Woburn' 'Leaside'
 'Central Bay Street' 'Christie' 'Cedarbrae' 'Hillcrest Village'
 'Bathurst Manor' 'Downsview North' 'Wilson Heights' 'Thorncliffe Park'
 'Adelaide' 'King' 'Richmond' 'Dovercourt Village' 'Dufferin'
 'Scarborough Village' 'Fairview' 'Henry Farm' 'Oriole' 'Northwood Park'
 'York University' 'East Toronto' 'Harbourfront East' 'Toronto Islands'
 'Union Station' 'L

Get rid of rowns where Borough is not assigned

In [10]:
# get rid of rows where the Borough is not assigned
print(df.shape)
df=df[df['Borough']!='Not assigned']
print(df.shape)
print(df['Borough'].unique())

(288, 3)
(211, 3)
['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'Central Toronto'
 'Mississauga']


If a neighborhood is not assigned, change it to match the borough name

In [11]:
#If a neighborhood is not assigned, change it to be the same as the borough
df.Neighborhood.value_counts()['Not assigned'] #how many do we have?
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']
df[0:10] #you can see that the Queen's Park neighborhood was assigned

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [12]:
all_postalcodes = df.PostalCode.unique()
len(all_postalcodes)

103

combine multiple rows for a postalcode - combine the neighborhoods into commas separated lists

In [13]:
for postalcode in all_postalcodes:
    #print('\n\npostalcode=', postalcode, '\n')
    neighborhoods_to_combine = df[df['PostalCode']==postalcode].Neighborhood.unique()
    number_of_neighborhoods_to_combine = neighborhoods_to_combine.size
    neighborhoods_to_combine_csv_str = ', '.join(map(str, neighborhoods_to_combine))
                                      
    #print(number_of_neighborhoods_to_combine, ' - ', neighborhoods_to_combine_csv_str)
    #rows = df[df['PostalCode']==postalcode]
    #rows.Neighborhood=neighborhoods_to_combine_csv_str
    df.loc[df.PostalCode == postalcode, 'Neighborhood'] = neighborhoods_to_combine_csv_str
    
    #print(rows)
    #new_df.append(rows)


Combined the neighborhoods by postalcode, but didn't get rid of duplicate rows generated yet - let's get rid of duplicates now

In [14]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)
df

(211, 3)
(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


fix the index

In [15]:
df.reset_index(inplace=True, drop=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Last step for part 1 - print the datafram shape

In [16]:
df.shape

(103, 3)